In [ ]:
if True:
    from julia.api import Julia
    jl = Julia(compiled_modules=False)

import julia; julia.install(quiet=True)
from julia import Main

import holoviews as hv
hv.extension('plotly', logo=False)
import panel

import numpy as np

%load_ext julia.magic

In [2]:
%%julia
using LinearAlgebra, Latexify;

<div style="float:center;width:100%;text-align: center;"><strong style="height:60px;color:darkred;font-size:40px;">Coordinate Systems</strong></div>

# 1. Substituting Linear Expressions: Matrix Multiplication

## 1.1 A Simple Example

Consider the following set of Linear Equations:

$\qquad
(\xi) \Leftrightarrow \left\{ \quad \begin{align*}
2 x_1 - 3 x_2 + 2 x_3 = \;7 \\
3 x_1 - 2 x_2 + 3 x_3 = 13 \\
2 x_1 + 4 x_2 - 2 x_3 = \;6 
\end{align*} \quad \right\} \Leftrightarrow \;
\left(
\begin{array}{rrr}
2 & -3 & 2 \\
3 & -2 & 3 \\
2 & 4 & -2 \\
\end{array}
\right)
\begin{pmatrix} x_1 \\ x_2 \\ x_3 \end{pmatrix}
= \left( \begin{array}{r} 7 \\ 13 \\ 6 \end{array} \right)
$

Let's change to **different variables**, e.g.,

$\qquad \left. \begin{align}
    & x_1 = \; \tilde{x_1} +\ 3 \tilde{x_2} - \; \tilde{x_3} \\
    & x_2 = 2 \tilde{x_1} +  2 \tilde{x_2} + 4 \tilde{x_3} \\
    & x_3 = - \tilde{x_1} + \; \tilde{x_2} + \; \tilde{x_3}
\end{align}
\; \right\}
 \Leftrightarrow \; \begin{pmatrix} x_1 \\ x_2 \\ x_3 \end{pmatrix} =
\left(\begin{array}{rrr}
1 & 3 & -1 \\
2 & 2 & 4 \\
-1 & 1 & 1 \\
\end{array}\right)
\left( \begin{array}{r} \tilde{x_1} \\ \tilde{x_2} \\ \tilde{x_3} \end{array} \right)
$

We can **substitute the change of variables** equations for $x_1, x_2$ and $x_3$ in the original system of linear equations and simplify, e.g., the first equation becomes

$
\qquad \begin{align}
2 x_1 - 3 x_2 + 2 x_3 = 7 
& \quad \Rightarrow \; 2 \left(   \tilde{x_1} + 3 \tilde{x_2} - \tilde{x_3}\right)
                              - 3 \left( 2 \tilde{x_1} +  2 \tilde{x_2} + 4 \tilde{x_3}  \right)  
                              + 2 \left( - \tilde{x_1} + \tilde{x_2} + \tilde{x_3} \right) = 7 \\
& \quad \Leftrightarrow \; -6 \tilde{x_1} + 2  \tilde{x_2} -12  \tilde{x_3} = 7
\end{align}
$

In terms of the matrix equations, this is

$\qquad (\xi) \Leftrightarrow \quad
\left(
\begin{array}{rrr}
2 & -3 & 2 \\
3 & -2 & 3 \\
2 & 4 & -2 \\
\end{array}
\right)
\left(\begin{array}{rrr}
1 & 3 & -1 \\
2 & 2 & 4 \\
-1 & 1 & 1 \\
\end{array}\right)
\left( \begin{array}{r} \tilde{x_1} \\ \tilde{x_2} \\ \tilde{x_3} \end{array} \right)
= \left( \begin{array}{r} 7 \\ 13 \\ 6 \end{array} \right)
$

**The simplification step turns out to be matrix multiplication.**

Carrying out the matrix multiplication, we obtain the system of equations

$\qquad (\xi) \Leftrightarrow \quad
\left(
\begin{array}{ccc}
-6 & 2 & -12 \\
-4 & 8 & -8 \\
12 & 12 & 12 \\
\end{array}
\right)
\left( \begin{array}{r} \tilde{x_1} \\ \tilde{x_2} \\ \tilde{x_3} \end{array} \right)
= \left( \begin{array}{r} 7 \\ 13 \\ 6 \end{array} \right)
$

## 1.2. The General Case

This holds true in general:

Given a system of linear equations $(\xi) \Leftrightarrow \; A x = b$<br>
and a change of variables $x = S \tilde{x}$,<br>
the system in written in terms of the new variables is given by $(\xi) \Rightarrow A S\ \tilde{x} = b$ 

**Remark:** For the new system to be equivalent to the original system,<br>
we need to be able to reverse the substitution, i.e.,<br>
we need $S$ to be invertible:

$\qquad
x = S \tilde{x} \Leftrightarrow \tilde{x} = S^{-1} x
$

Applying this to the substitution equation above, we have

$\qquad
A x = b \; \Rightarrow A S\ \tilde{x} = b \Rightarrow A S\ S^{-1} x = b \Rightarrow A x = b
$

# 2. The Column View of A x = b

## 2.1 Decompose a Vector

Consider the column view of our example system

$\qquad
(\xi) \Leftrightarrow \quad
\left( \begin{array}{r} 7 \\ 13 \\ 6 \\ \end{array} \right) =
x_1 \left( \begin{array}{r} 2 \\ 3 \\ 2 \\ \end{array} \right) +
x_2 \left( \begin{array}{r} -3 \\ -2 \\ 4 \\ \end{array} \right) +
x_3 \left( \begin{array}{r} 2 \\ 3 \\ -2 \\ \end{array} \right)
$

Looked at this way, our problem $A x = b$<br><br>
decomposes the vector
$b = \left(\begin{array}{c} 7 \\ 13 \\ 6 \end{array} \right)$ 
into a **linear combination**  of the three vectors<br><br>$\qquad$
$a_1 = \left( \begin{array}{r} 2 \\ 3 \\ 2 \\ \end{array} \right)$,
$a_2 = \left( \begin{array}{r} -3 \\ -2 \\ 4 \\ \end{array} \right)$ and
$a_3 = \left( \begin{array}{r} 2 \\ 3 \\ -2 \\ \end{array} \right)$

That is to say, **to reach the vector** $\mathbf{b}$, walk a distance $x_1$ along vector $a_1$,<br>
followed by a distance $x_2$  in the direction of the vector $a_2$<br>
followed by a distance $x_3$ in the direction of $a_3$.

Incidentally, the solution of our $A x = b$ problem is $x_1=3, x_2=1$ and $x_3=2$.

Let's fix the vectors $a_1, a_2$ and $a_3$, and consider other vectors $b$,
e.g., $b = \left( \begin{array}{r} 9 \\ 11 \\ -2 \\ \end{array} \right)$.

Solving our $A x = b$ problem, we get

$\qquad
\left( \begin{array}{r} 9 \\ 11 \\ -2 \\ \end{array} \right) =
2 \left( \begin{array}{r} 2 \\ 3 \\ 2 \\ \end{array} \right)
-1 \left( \begin{array}{r} -3 \\ -2 \\ 4 \\ \end{array} \right) +
1 \left( \begin{array}{r} 2 \\ 3 \\ -2 \\ \end{array} \right)
$

___
We can think of our vectors $a_1, a_2$ and $a_3$ as a **system of coordinate vectors**<br>
and the solution $x, y$ and $z$ of $b = x_1 a_1 + x_2 a_2 + x_3 a_3$<br> as the **coordinates** of $b$ with respect to the coordinate axis vectors $a_1, a_2, a_3$.

## 2.2 Example Computation

In [3]:
%%julia
a_1 = [ 2, 3, 2]
a_2 = [-3,-2, 4]
a_3 = [ 2, 3,-2]
A   = [ a_1 a_2 a_3 ]
b   = [7, 13, 6]

z   = [0, 0, 0]
ZA  = [z a_1 z a_2 z a_3]'          # coordinate axes
ZAb = [ z 3a_1 z 1a_2 z 2a_3]'      # distance along coordinate axes (decomposition of b)
Zb = [z b]'                         # b vector
;

In [4]:
# Plot x a_1, y a_2, z a_3 decomposition of b
h_axes =\
hv.Path3D( Main.ZA[0:2,:]).opts( color="red", line_width=8)*\
hv.Path3D( Main.ZA[2:4,:]).opts( color="blue", line_width=8)*\
hv.Path3D( Main.ZA[4:6,:]).opts( color="green", line_width=8)

h_walk =\
hv.Path3D( Main.ZAb[0:2,:]).opts( color="red", line_width=3)*\
hv.Path3D( Main.ZAb[2:4,:]).opts( color="blue", line_width=3)*\
hv.Path3D( Main.ZAb[4:6,:]).opts( color="green", line_width=3)

h_b =\
hv.Path3D( Main.Zb).opts( color="black", line_width=3)

h_axes*h_walk*h_b

:Overlay
   .Path3D.I   :Path3D   [x,y,z]
   .Path3D.II  :Path3D   [x,y,z]
   .Path3D.III :Path3D   [x,y,z]
   .Path3D.IV  :Path3D   [x,y,z]
   .Path3D.V   :Path3D   [x,y,z]
   .Path3D.VI  :Path3D   [x,y,z]
   .Path3D.VII :Path3D   [x,y,z]

In [5]:
%%julia
x = A\b

walk=[z (x[1]*a_1) (x[1]*a_1+x[2]*a_2)  (x[1]*a_1+x[2]*a_2+x[3]*a_3)]'
;

In [6]:
# show the walk to reach b
h_axes*h_b*\
hv.Path3D(Main.walk).opts(title="Walked Path")

:Overlay
   .Path3D.I   :Path3D   [x,y,z]
   .Path3D.II  :Path3D   [x,y,z]
   .Path3D.III :Path3D   [x,y,z]
   .Path3D.IV  :Path3D   [x,y,z]
   .Path3D.V   :Path3D   [x,y,z]

## 2.3 Conclusion

The column view makes it clear that solving the equation $A x = b$<br>
**decomposes the** $\mathbf{b}$ **vector** into components along each of the columns of $A$.

**Remark:** the vectors are specified in terms of the standard basis, e.g.

$\qquad
b = \left( \begin{array}{r} 7 \\ 13 \\ 6 \end{array} \right) =
   7 \left( \begin{array}{r} 1 \\ 0 \\ 0 \end{array} \right) +
  13 \left( \begin{array}{r} 0 \\ 1 \\ 0 \end{array} \right) +
  6 \left( \begin{array}{r} 0 \\ 0 \\ 1 \end{array} \right)
$

The same observation holds for $a_1, a_2$ and $a_3$.

**Important Remarks:**
* As long as we agree on the coordinate vectors,<br> **we only need to specify** $\mathbf{x_1, x_2}$ and $\mathbf{x_3}$ to determine the vector $b$.<br><br>
* We do not need to explicitely specify the coordinate system $a_1, a_2, a_3$.<br>
**The coordinate vector** $\mathbf{(x_1, x_2, x_3 )}$ **represents that vector** $b$.<br><br>
* We would like this coordinate vector representation to be unique:<br>
$A x = b$ should not have free variables, i.e., **the coordinate axis vectors** $\mathbf{ \left\{ a_1, a_2, a_3 \right\}}$ **should be linearly independent.**<br><br>
Terminology: a set of linearly independent coordinate system vectors ($\left\{ a_1, a_2, a_3 \right\}$ in our example) is called a **basis**.

# 3. Changing Coordinate Systems

## 3.1 Just another Coordinate System

As the previous section makes clear, the vector $b$ is originally specified in terms of the standard basis.

**Remark:** Actually, $b$ could be a coordinate vector with respect to some unmentioned coordinate system.<br>$\qquad$
Each of the standard basis vectors than stands for the actual basis vector.

The basis vectors $a_1, a_2$ and $a_3$ that we used is not very nice: could we change it?

We would like to write $b$ with respect to a **different coordinate system** $\mathbf{s_1, s_2}$ **and** $\mathbf{s_3}$,<br>
i.e.,  $b = \tilde{x_1} s_1 + \tilde{x_2} s_2 + \tilde{x_3} s_3$.<br>
In other words, we find the coordinate vector $\tilde{x}$ by solving the equation $S x = b$, with $S=\left( s_1 \; s_2 \; s_3 \right)$.

Note that the $s_1, s_2$ and $s_3$ vectors are expressed with respect to the standard basis.

## 3.2 Relation between Coordinate Vectors

How do the coordinate vectors $(x, y, z)$ and $(\tilde{x_1}, \tilde{x_2}, \tilde{x_3})$ relate?

We have $A x = b$ and $S \tilde{x} = b$. Eliminating $b$, we see that $A x = S \tilde{x}$,<br>
where $A = ( a_1 \; a_2 \; a_3 )$ and $S = (s_1 \; s_2 \; s_3 )$ are invertible matrices<br>
(we choose our coordinate vectors to be linearly independent).

Solving for $x$, we find $x = A^{-1} S \tilde{x}$

If we set $\tilde{S} = A^{-1} S$, we see that we express the coordinate vector $x$ (the $b$ vector with respect to the coordinate system $a_1,a_2,a_3)$)<br>
in terms of the coordinate vectors formed by the columns of $\tilde{S}$.

Let's look at the first column $\tilde{s_1} = A^{-1} s_1 \Leftrightarrow s_1 = A \tilde{s_1}$.
We recognize this: $\tilde{s_1}$ is the coordinate vector $s_1$ with respect to the coordinate system $\left\{ a_1\; a_2 \; a_3 \right\}$.

A similar analysis holds for the remaining vectors $\tilde{s_2}$ and $\tilde{s_3}$.

We have
* $b \xrightarrow{b = A x} x$, where $b$ is the coordinate vector with respect to the standard basis<br>
$\qquad$ and $x$ is the coordinate vector with respect to the basis formed by the columns of $A$.<br>
The columns of $A$ are the coordinate axis vectors with respect to the standard basis.<br><br>
* $b \xrightarrow{b = S \tilde{x}} \tilde{x}$, where $b$ is the coordinate vector with respect to the standard basis<br>
$\qquad$ and $\tilde{x}$ is the coordinate vector with respect to the basis formed by the columns of $S$.<br>
$\qquad$ The columns of $S$ are the coordinate axis vectors with respect to the standard basis.<br><br>
* $x \xrightarrow{x = \tilde{S} \tilde{x}} \tilde{x}$, where $x$ is the coordinate vector with respect to the basis formed by the columns of $A$<br>
$\qquad$ and the $\tilde{s}_i$ vectors are the coordinate vectors of the $s_i$ with respect to the basis formed by the columns of $A$<br>
$\qquad$ In turn $\tilde{x}$ is the coordinate vector with respect to the basis $\left\{ \tilde{s}_1, \tilde{s}_2, \tilde{s}_3 \right\}$,<br>


# 4. Suggestion

The QR algorithm computes an orthonormal basis $\left\{ q_i \right\}_{i = 1,2,3}$ from the basis $\left\{ a_i  \right\}_{i = 1,2,3}$

Write the example code to express $b$ with respect to the $\left\{ q_i \right\}_{i = 1,2,3}$<br>
starting from $b$ and starting from $x$.